# Convolutional Neural Network Classification
We want to classify an image this time around

As usual import the required dependencies is the first step¶


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

## Again creating a neural network class but this time with convolutions
* https://pytorch.org/docs/stable/nn.html#id21


Conv2D() requires: in_ch, out_ch, kernel_size, stride
https://pytorch.org/docs/0.4.1/nn.html?highlight=conv2d#torch.nn.Conv2d

Let's start with the following layers:
* (look at forward first)
* a conv layer
    * with relu activation and max pooling (2x2)
    * a good guess would be about 20 features (5x5 kernel)
* a second conv layer, relu, 2x2 maxpooling
    * with relu activation and max pooling (2x2)
    * a good guess would be about 50 features (5x5 kernel)

* going to 500 neuron fully connected, with relu
* before going down to the 10 output neurons (for the digits 0-9)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(WE WILL GET A GRAY IMAGE, HOW MANY FEATURE MAPS, 5, 1)
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(AND THEN WE HAVE XXXX, 5, 1)
        self.pool2 = torch.nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)   


In [ ]:
def train(model, device, train_loader, optimizer, epoch, log_interval=1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            print(data.size())
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### GPU support
if you have a gpu in your computer that will make a huge difference!

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

### Dataset
MNIST is an old dataset it's not too large

you can download it directly from within pytorch

there is a batch_size parameter! 

In [ ]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True)

In [ ]:
for epoch in range(1, 2):
        train(model, device, train_loader, optimizer, epoch)

## ALWAYS HAVE A SEPARATE VALIDATION SET

In [ ]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False,
                       # transforming can also be dataaugmentation
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])
),
        batch_size=64, shuffle=True)

How well do we perform on the validation one!?

In [ ]:
test(model, device, test_loader)

what happened? can we visualise that?


In [ ]:
train_loader.dataset

In [ ]:
(image, label) = train_loader.dataset.__getitem__(2)
print(image.size())

In [ ]:
image=image.reshape([28,28])
plt.gray()
plt.imshow(image)

ahhh, so that's what they look like!

In [ ]:
# what's the label for this image
print("Number: %d" % label.item())

In [ ]:
# can we load our own image and test it as well?

In [ ]:
im_test = Image.open('test.0.dark.png')

In [ ]:
print(im_test)

In [ ]:
trans = transforms.ToPILImage()
trans1 = transforms.ToTensor()
gray = transforms.Grayscale()
plt.imshow(trans(trans1(im_test.convert("RGB"))))

In [ ]:
input_test = trans1(gray(im_test))

### what does out network say to that new image!

In [ ]:
pred = model(input_test.reshape(1, 1, 28, 28))

In [ ]:
# input_test, target = input_test.to(device), target.to(device)
model.eval()

In [ ]:
print(pred)

In [ ]:
pred

In [ ]:
pred.argmax()

In [ ]:
# Did it work for you? 
# try again if not?